# Types and Dispatch in Julia

Julia is built around types.

Software architectures in Julia are built around good use of the type system.

# Abstract vs concrete types

*Concrete types* are the types of objects. They specify the data structure of an object.

*Abstract types* cannot be instantiated. They define sets of related concrete types (their descendants) by their behavior.

In [ ]:
typeof(3)

In [ ]:
typeof(3.0)

In [ ]:
isconcretetype(Float64)

In [ ]:
isabstracttype(Number)

In [ ]:
isabstracttype(Real)

### Duck typing

A `Number` is some abstract type that can do things like `+`,`-`,`*`, and `/`. In this category we have (concrete) things like `Float64` and `Int32`.

An `AbstractArray` is a type that can be indexed like `A[i]`. An `AbstractArray` may be mutable, meaning it can be set: `A[i]=v`.

### Inspecting the type tree

In [ ]:
supertype(Float64)

In [ ]:
supertype(AbstractFloat)

In [ ]:
subtypes(AbstractFloat)

In [ ]:
supertype(Real)

In [ ]:
supertype(Number)

Everything is a subtype of `Any`

In [ ]:
Number <: Any

In [ ]:
Float64 <: Any

In [ ]:
Int32 <: Any

In [ ]:
Int32 <: String

There is also `isa` for objects:

In [ ]:
3.0 isa Float64

In [ ]:
3 isa Float64

In [ ]:
typeof(3) <: Float64

We define a function that, given a concrete type `T`, prints the single branch of the type tree that leads from the top node `Any` to the leave `T`.

In [ ]:
function show_supertypes(T) 
 print(T)
 while T != Any 
     T = supertype(T) 
     print(" <: ", T) 
 end 
end

In [ ]:
show_supertypes(Float64)

In [ ]:
show_supertypes(String)

Let's extract a bunch of branches

In [ ]:
function show_subtypetree(T, level=1, indent=4)
   level == 1 && println(T)
   for s in subtypes(T)
     println(join(fill(" ", level * indent)) * string(s))
     show_subtypetree(s, level+1, indent)
   end
end

In [ ]:
show_subtypetree(Number)

Note that concrete types are the leaves of the type tree.

Abstract types are nodes in the type graph.

# Functions, Methods, and Dispatch

Let's define a *function* that calculates the absolute value of a number (like Julias `abs` already does).

How would we practically calculate the absolute values of the numbers $-4.32$ and $1.0 + 1.0i$?

Presumably:
* Real number: "Drop the sign." => `myabs(-4.32) = 4.32`
* Complex number: "Square root of z times the complex conjugate of z." => `myabs(1.0 + 1.0im) = sqrt(2) ≈ 1.414`

We see that the *methods* that we use depend on the type of the number.

While the single **function** represents the *what* ("calculate the absolute value"), there might be different **methods** describing the *how*.

We can use the `::` operator to annotate function arguments with types and define different methods.

In [ ]:
myabs(x::Float64) = sign(x) * x

In [ ]:
myabs(-4.32)

In [ ]:
myabs(1.0 + 1.0im)

In [ ]:
myabsthatdoesntexist(1.0 + 1.0im)

In [ ]:
myabs(z::ComplexF64) = sqrt(real(z * conj(z)))

In [ ]:
myabs(1.0 + 1.0im)

In [ ]:
methods(myabs)

One can check which particular method is being used through the `@which` macro.

In [ ]:
@which myabs(-4.32)

In [ ]:
@which myabs(1.0 + 1.0im)

Note that we should better loosen our type restrictions:

In [ ]:
myabs(-3)

In [ ]:
myabs(1 + 1im)

In [ ]:
myabs(x::Real) = sign(x) * x
myabs(z::Complex) = sqrt(real(z * conj(z)))

In [ ]:
myabs(-3)

# Multiple Dispatch

Julia's dispatch mechanism always chooses the *most specific method* for the given input types.

In [ ]:
f(a, b::Any)              = "fallback"
f(a::Number, b::Number)   = "a and b are both numbers"
f(a::Number, b)           = "a is a number"
f(a, b::Number)           = "b is a number"
f(a::Integer, b::Integer) = "a and b are both integers"

In [ ]:
methods(f)

In [ ]:
f(1.5, 2)

In [ ]:
f(1, "Köln!")

In [ ]:
f(1, 2)

In [ ]:
f("Hello", "World!")

In [ ]:
@which f(1, 2)

In [ ]:
@which f(1, "Köln!")

In [ ]:
methods(+)

In [ ]:
@which true + false

In [ ]:
@which "Hello"*"World!"

Julia's standard functions are not special by any means.

We can easily modify or add methods to them as well.

In [ ]:
import Base: + # we have to import functions to override/extend them
+(x::String, y::String) = x * " " * y

In [ ]:
"Kölle" + "Alaaf"

Any function based on the `+` operation can now handle `String`s as well.

In [ ]:
sum(["This", "works", "although", "we", "never", "touched", "sum!"])

It happens rarely, but it can happen that there is no unique most specific method:

In [ ]:
f(x::Int, y::Any) = println("int")
f(x::Any, y::String) = println("string")
f(3, "test")

# Parametric types

Types can have *type parameters*. The most prominent example is Julia's array type.

In [ ]:
M = rand(2,2)

In [ ]:
typeof(M)

Here, `Array` is a parametric array datatype. Its type parameters `Float64` and `2` indicate the type of the element the array can hold and and its dimensionality. Hence, we have a matrix of floating point numbers.

This generalizes as expected. Here, a matrix of `String`s:

In [ ]:
M = fill("Cologne", 2,2)

In [ ]:
eltype(M)

We can also nest parametric types. This is a vector of matrices of `Float64`s.

In [ ]:
v = [rand(2,2) for i in 1:3]

In [ ]:
eltype(v)

Since vectors and matrices pop up so frequently, Julia has a nice shortcut type alias for them.

In [ ]:
Vector{Float64} === Array{Float64, 1}

In [ ]:
Matrix{Float64} === Array{Float64, 2}

Another example of a parametric type is the `Tuple`.

In [ ]:
(1,2.0,"3")

In [ ]:
typeof((1,2.0,"3"))

### `UnionAll` types and `where`

Note that parametric types have the following (somewhat counterintuitive at first) property

In [ ]:
Vector{Float64} <: Vector{Real}

although we have

In [ ]:
Float64 <: Real

The reason is that `Vector{Real}` is a concrete type - it describes a vector of values that individually have a type `T <: Real`  - and concrete types don't have subtypes.

In [ ]:
isconcretetype(Vector{Real})

In [ ]:
Real[1, 2.2, 13f0]

What we actually *mean* is

In [ ]:
Vector{Float64} <: Vector{T} where T<:Real

Here, `Vector{T} where T <: Real` describes the *set* of concrete `Vector` types whose elements are of any single type `T` that is a subtype of `Real`.

In [ ]:
Vector{Int64} <: Vector{T} where T<:Real

In [ ]:
Vector{Real} <: Vector{T} where T<:Real

Using this notation, our `Vector{Real}` from above can more explicitly be written as `Vector{T where T<:Real}`

In [ ]:
Vector{Real} === Vector{T where T<:Real}

### Type parameters in function signatures

In [ ]:
h(x::Integer) = typeof(x)

In [ ]:
h(x::T) where T = T

**Quick exercise**: Write a single-argument function that takes any real matrix as input and returns the element type of the matrix.

**Solution:**
<details>
  <summary>Click to reveal</summary>
<br>
    
```julia
g(x::Matrix{T}) where T<:Real = T
```

or alternatively

```julia
g(x::Matrix{<:Real}) = eltype(x)
```
</details>

**Test:**

In [ ]:
g(rand(Float32,2,2))

In [ ]:
g(rand(Int16,2,2))

# "Diagonal" dispatch

In [ ]:
d(x::T, y::T) where T = "same type"
d(x, y) = "different types"

In [ ]:
d(3, 4)

In [ ]:
d(3.0, 1.0)

In [ ]:
d(1, 4.2)

# Duck typing examples

### `UnitRange`

In [ ]:
x = 1:30

In [ ]:
typeof(x)

In [ ]:
typeof(x) <: AbstractArray

Because it is a subtype of `AbstractArray` we can do array-like things with it (it should basically behave like an array!)

In [ ]:
x[3]

In [ ]:
size(x)

In [ ]:
eltype(x)

However, it's not implemented like a regular `Array` at all.

In fact, it's just two numbers! We can see this by looking at it's fields:

In [ ]:
fieldnames(typeof(x))

or just by inspecting the source code

In [ ]:
@which UnitRange(1, 30)

It is an `immutable` type which just holds the start and stop values.

This means that indexing, `A[i]`, is not just a look-up but a (small) function (try `@which getindex(x, 4)`).

What's nice about this is that we can use it in calculations and no array, containing the numbers from 1 to 30, is ever created.

Allocating memory is typically costly.

In [ ]:
@time collect(1:10000000);

But creating an immutable type of two numbers is essentially free, no matter what those two numbers are:

In [ ]:
@time 1:10000000;

Yet, in code they *act* the same way.

# Other types

* Union types: `Union{Float64, Int32}`
* [Bitstypes](https://docs.julialang.org/en/v1/manual/calling-c-and-fortran-code/#man-bits-types-1) (check with `isbits(x)`, `isbitstype(T)`)
* [Value types](https://docs.julialang.org/en/v1/manual/types/#%22Value-types%22-1) (allows dispatch on values)

See https://docs.julialang.org/en/latest/manual/types/ for more.

# Extra: slurping and splatting

In [ ]:
f(x...) = println(x) # slurping

In [ ]:
f(3, 1.2, "Carsten")

In [ ]:
g(x::Vector) = +(x...) # splat vector into addition operation

In [ ]:
g([1,2,3])

# Core messages of this Notebook

* **Concrete types** describe data structures, i.e. concrete implementations.
* **Abstract types** define the kind of a thing (What is it? What can I do with it?), i.e. an informal interface. This is also known as **duck-typing**.
* A **function** (the what) can have multiple **methods** (the how).
* **Multiple dispatch**: Julia selects the method to run based on the types of all input arguments and chooses the most specialized one.
* Types can have parameters, i.e. `Vector{Float64}`. We can use the notation `T where T<:SomeSuperType` to address *sets* of types.